# **Generación dataset del Nivel 2**

En este notebook generamos el dataset del Nivel 2, que consiste en preguntas/respuestas que requieren cálculos relativamente sencillos.

Comenzamos cargando el dataset original, e inspeccionándolo.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import random
import itertools

In [ ]:
import json
from typing import Dict, List, Optional, Tuple

In [ ]:
strings = {'Sección' : 'str', 'cod_ccaa' : 'str', 'cod_prov' : 'str', 'cod_mun' : 'str', 'cod_sec' : 'str'}

In [ ]:
df_eleccion = pd.read_csv('/content/drive/MyDrive/Practica_LLM_Engineering_25/df_elecciones_19.csv', dtype = strings)

In [ ]:
df_eleccion.head()

,Elecciones,Sección,cod_ccaa,cod_prov,cod_mun,cod_sec,CCAA,Provincia,Municipio,Distrito,...,Renta persona 2017,Renta persona 2015,Renta hogar 2017,Renta hogar 2015,Renta Salarios 2018,Renta Salarios 2015,Renta Pensiones 2018,Renta Pensiones 2015,Renta Desempleo 2018,Renta Desempleo 2015
0,Abril 2019,022019041010400101001,01,04,04001,0400101001,Andalucía,Almería,Abla,1,...,9159.0,8788.0,20172.0,19546.0,5574.0,4833.0,3286.0,3082.0,403.0,471.0
1,Abril 2019,022019041010400201001,01,04,04002,0400201001,Andalucía,Almería,Abrucena,1,...,8827.0,8107.0,17841.0,17115.0,4640.0,4048.0,3418.0,2770.0,568.0,620.0
2,Abril 2019,022019041010400301001,01,04,04003,0400301001,Andalucía,Almería,Adra,1,...,8965.0,8267.0,26498.0,24688.0,5121.0,4795.0,2499.0,2301.0,337.0,333.0
3,Abril 2019,022019041010400301002,01,04,04003,0400301002,Andalucía,Almería,Adra,1,...,8599.0,7941.0,25677.0,23400.0,5381.0,4837.0,1815.0,1724.0,343.0,464.0
4,Abril 2019,022019041010400301003,01,04,04003,0400301003,Andalucía,Almería,Adra,1,...,8076.0,7150.0,22051.0,19687.0,5224.0,4044.0,1170.0,1198.0,416.0,476.0


In [ ]:
df_eleccion.columns

Index(['Elecciones', 'Sección', 'cod_ccaa', 'cod_prov', 'cod_mun', 'cod_sec',
       'CCAA', 'Provincia', 'Municipio', 'Distrito', 'Censo_Esc',
       'Votos_Total', 'Participación', 'Nulos', 'Votos_Válidos', 'Blanco',
       'V_Cand', 'PP', 'PSOE', 'Cs', 'UP', 'IU', 'VOX', 'UPyD', 'MP', 'CiU',
       'ERC', 'JxC', 'CUP', 'DiL', 'PNV', 'Bildu', 'Amaiur', 'CC', 'FA', 'TE',
       'BNG', 'PRC', 'GBai', 'Compromis', 'PACMA', 'Otros', '% PP', '% PSOE',
       '% UP', '% VOX', '% Cs', '% IU', 'Ganador', 'Segundo', 'Tercero',
       'Cuarto', 'Quinto', 'edad_0-4', 'edad_5-9', 'edad_10-14', 'edad_15-19',
       'edad_20-24', 'edad_25-29', 'edad_30-34', 'edad_35-39', 'edad_40-44',
       'edad_45-49', 'edad_50-54', 'edad_55-59', 'edad_60-64', 'edad_65-69',
       'edad_70-74', 'edad_75-79', 'edad_80-84', 'edad_85-89', 'edad_90-94',
       'edad_95-99', 'edad_100 y más', 'Población Total', 'Hombres', 'Mujeres',
       '% mayores 65 años', '% 20-64 años', '% menores 19 años',
       'Afiliados S

En este caso, la generación de preguntas y respuestas la incluimos en una sola clase. Esta se compone en realidad de 8 métodos, cada uno de los cuales genera un tipo de pregunta/respuesta. Incluyen comparaciones temporales entre elecciones, comparaciones geográficas, operaciones estadísticas basicas, rankings, etc...

La clase también incluye un método para generar preguntas/respuestas en función de los pesos que queramos dar a cada uno de los métodos.

In [ ]:
class GeneradorPreguntasNivel2:
    def __init__(self, df: pd.DataFrame):
        """
        Inicializa el generador con el dataframe de datos electorales

        Args:
            df: DataFrame con columnas como:
                - Sección, Municipio, Provincia, CCAA
                - % PP, % PSOE, % VOX, % Cs, % UP, etc.
                - Participación, Censo_Esc, Votos_Total
                - Variables socioeconómicas
                - Año (si tienes múltiples datasets)
        """
        self.df = df

        # Extraer entidades para variabilidad
        self.secciones = df['cod_sec'].unique().tolist()
        self.municipios = df['Municipio'].unique().tolist()
        self.provincias = df['Provincia'].unique().tolist()
        self.comunidades = df['CCAA'].unique().tolist()

        # Partidos políticos (ajusta según tus columnas)
        self.partidos = ['PP', 'PSOE', 'VOX', 'Cs', 'UP', 'IU']

        # Variables socioeconómicas disponibles
        self.variables_socio = [
            'Renta persona 2017', 'Renta hogar 2017',
            'Renta Salarios 2018', 'Renta Pensiones 2018',
            '% mayores 65 años', '% 20-64 años', '% Paro s/ Afiliados SS Municipio',
            '% Afiliados SS autónomos', 'Renta Desempleo 2018'
        ]

        # Columnas electorales
        self.columnas_electorales = ['% ' + p for p in self.partidos] + ['Participación']

        # Si tienes múltiples años
        if 'Elecciones' in df.columns:
            self.elecciones = sorted(df['Elecciones'].unique().tolist())
        else:
            self.elecciones = ['Abril 2019']  # Asume Abril 2019 si no hay columna año

    def generar_evolucion_temporal(self) -> Optional[Dict]:
        """
        Tipo 1: Evolución entre años para un municipio/partido
        """
        if len(self.elecciones) < 2:
            return None

        municipio = random.choice(self.municipios)
        partido = random.choice(self.partidos)
        eleccion1, eleccion2 = random.sample(self.elecciones, 2)

        # Obtener datos reales
        datos_eleccion1 = self.df[(self.df['Municipio'] == municipio) &
                            (self.df['Elecciones'] == eleccion1)]
        datos_eleccion2 = self.df[(self.df['Municipio'] == municipio) &
                            (self.df['Elecciones'] == eleccion2)]

        if len(datos_eleccion1) == 0 or len(datos_eleccion2) == 0:
            return None

        columna = f'% {partido}'
        if columna not in datos_eleccion1.columns or columna not in datos_eleccion2.columns:
            return None

        valor1 = 100 * datos_eleccion1[columna].mean()
        valor2 = 100 * datos_eleccion2[columna].mean()
        cambio = valor2 - valor1
        cambio_porcentual = (cambio / valor1 * 100) if valor1 != 0 else 0

        pregunta = f"¿Cómo evolucionó el porcentaje de voto a {partido} en {municipio} entre {eleccion1} y {eleccion2}?"

        respuesta = f"El {partido} en {municipio} pasó de {valor1:.2f}% en {eleccion1} a {valor2:.2f}% en {eleccion2}. "
        if cambio > 0:
            respuesta += f"Esto representa un incremento de {abs(cambio):.2f} puntos porcentuales "
            respuesta += f"({abs(cambio_porcentual):.1f}% más)."
        elif cambio < 0:
            respuesta += f"Esto representa una disminución de {abs(cambio):.2f} puntos porcentuales "
            respuesta += f"({abs(cambio_porcentual):.1f}% menos)."
        else:
            respuesta += "No hubo cambio en el porcentaje de voto."

        return {
            "instruction": pregunta,
            "input": f"Municipio: {municipio}, Partido: {partido}, Años: {eleccion1} y {eleccion2}",
            "output": respuesta,
            "metadata": {
                "tipo": "evolucion_temporal",
                "municipio": municipio,
                "partido": partido,
                "Elecciones": [eleccion1, eleccion2],
                "valores": [float(valor1), float(valor2)]
            }
        }

    def generar_comparacion_geografica(self) -> Optional[Dict]:
        """
        Tipo 2: Comparar municipios/provincias en un mismo año
        """
        tipo_comparacion = random.choice(['municipio', 'provincia', 'comunidad'])
        eleccion = random.choice(self.elecciones)
        variable = random.choice(self.columnas_electorales)

        datos_eleccion = self.df[self.df['Elecciones'] == eleccion] if 'Elecciones' in self.df.columns else self.df

        if tipo_comparacion == 'municipio':
            # Comparar 2 municipios de misma provincia
            provincia = random.choice(self.provincias)
            municipios_prov = self.df[self.df['Provincia'] == provincia]['Municipio'].unique()

            if len(municipios_prov) < 2:
                return None

            m1, m2 = random.sample(list(municipios_prov), 2)

            dato_m1 = 100 * datos_eleccion[datos_eleccion['Municipio'] == m1][variable].mean()
            dato_m2 = 100 * datos_eleccion[datos_eleccion['Municipio'] == m2][variable].mean()

            pregunta = f"¿En qué municipio de {provincia} fue mayor {variable} en {eleccion}: {m1} o {m2}?"
            respuesta = f"En {eleccion}, {variable} fue mayor en {m1 if dato_m1 > dato_m2 else m2}. "
            respuesta += f"Valores: {m1}: {dato_m1:.2f}%, {m2}: {dato_m2:.2f}%. "
            respuesta += f"Diferencia: {abs(dato_m1 - dato_m2):.2f} puntos."

            entidades = [m1, m2]

        elif tipo_comparacion == 'provincia':
            # Comparar 2 provincias
            p1, p2 = random.sample(self.provincias, 2)

            dato_p1 = 100 * datos_eleccion[datos_eleccion['Provincia'] == p1][variable].mean()
            dato_p2 = 100 * datos_eleccion[datos_eleccion['Provincia'] == p2][variable].mean()

            pregunta = f"¿En qué provincia fue mayor {variable} en la elección de {eleccion}: {p1} o {p2}?"
            respuesta = f"En {eleccion}, {variable} fue mayor en {p1 if dato_p1 > dato_p2 else p2}. "
            respuesta += f"Valores: {p1}: {dato_p1:.2f}%, {p2}: {dato_p2:.2f}%."

            entidades = [p1, p2]


        elif tipo_comparacion == 'comunidad':
            # Comparar 2 comunidades
            c1, c2 = random.sample(self.comunidades, 2)

            dato_c1 = 100 * datos_eleccion[datos_eleccion['CCAA'] == c1][variable].mean()
            dato_c2 = 100 * datos_eleccion[datos_eleccion['CCAA'] == c2][variable].mean()

            pregunta = f"¿En qué comunidad fue mayor {variable} en la elección de {eleccion}: {c1} o {c2}?"
            respuesta = f"En {eleccion}, {variable} fue mayor en {c1 if dato_c1 > dato_c2 else c2}. "
            respuesta += f"Valores: {c1}: {dato_c1:.2f}%, {c2}: {dato_c2:.2f}%."

            entidades = [c1, c2]



        return {
            "instruction": pregunta,
            "input": f"Año: {eleccion}, Variable: {variable}",
            "output": respuesta,
            "metadata": {
                "tipo": "comparacion_geografica",
                "subtipo": tipo_comparacion,
                "entidades": entidades,
                "variable": variable,
                "Elección": eleccion
            }
        }

    def generar_operacion_estadistica(self) -> Optional[Dict]:
        """
        Tipo 3: Promedio, máximo, mínimo, etc.
        """
        operacion = random.choice(['promedio', 'maximo', 'minimo', 'mediana'])
        ambito = random.choice(['municipio', 'provincia', 'comunidad'])
        variable = random.choice(self.columnas_electorales)
        eleccion = random.choice(self.elecciones)

        datos_eleccion = self.df[self.df['Elecciones'] == eleccion] if 'Elecciones' in self.df.columns else self.df

        if ambito == 'municipio':
            municipio = random.choice(self.municipios)
            datos = datos_eleccion[datos_eleccion['Municipio'] == municipio]
            texto_ambito = f"en el municipio de {municipio}"

        elif ambito == 'provincia':
            provincia = random.choice(self.provincias)
            datos = datos_eleccion[datos_eleccion['Provincia'] == provincia]
            texto_ambito = f"en la provincia de {provincia}"

        elif ambito == 'comunidad':
            comunidad = random.choice(self.comunidades)
            datos = datos_eleccion[datos_eleccion['CCAA'] == comunidad]
            texto_ambito = f"en la comunidad de {comunidad}"

        if len(datos) == 0:
            return None

        if operacion == 'promedio':
            valor = 100 * datos[variable].mean()
            pregunta = f"¿Cuál fue el {variable} promedio {texto_ambito} en {eleccion}?"
            respuesta = f"El {variable} promedio {texto_ambito} en {eleccion} fue de {valor:.2f}%."

        elif operacion == 'maximo':
            valor = 100 * datos[variable].max()
            # Encontrar sección con máximo
            seccion_max = datos.loc[datos[variable].idxmax(), 'Sección'] if 'Sección' in datos.columns else "una sección"
            pregunta = f"¿Cuál fue el {variable} máximo {texto_ambito} en {eleccion}?"
            respuesta = f"El {variable} máximo {texto_ambito} en {eleccion} fue de {valor:.2f}% en la sección ({seccion_max})."

        elif operacion == 'minimo':
            valor = 100 * datos[variable].min()
            seccion_min = datos.loc[datos[variable].idxmin(), 'Sección'] if 'Sección' in datos.columns else "una sección"
            pregunta = f"¿Cuál fue el {variable} mínimo {texto_ambito} en {eleccion}?"
            respuesta = f"El {variable} mínimo {texto_ambito} en {eleccion} fue de {valor:.2f}% en la sección ({seccion_min})."

        elif operacion == 'mediana':
            valor = 100 * datos[variable].median()
            pregunta = f"¿Cuál fue el {variable} mediano {texto_ambito} en {eleccion}?"
            respuesta = f"El {variable} mediano {texto_ambito} en {eleccion} fue de {valor:.2f}%."

        return {
            "instruction": pregunta,
            "input": f"Año: {eleccion}, Ámbito: {ambito}, Variable: {variable}",
            "output": respuesta,
            "metadata": {
                "tipo": "operacion_estadistica",
                "operacion": operacion,
                "ambito": ambito,
                "variable": variable,
                "año": eleccion,
                "valor": float(valor)
            }
        }

    def generar_diferencia_partidos(self) -> Optional[Dict]:
        """
        Tipo 4: Diferencia entre partidos en un mismo lugar/elección
        """
        ambito = random.choice(['municipio', 'provincia'])
        eleccion = random.choice(self.elecciones)

        datos_eleccion = self.df[self.df['Elecciones'] == eleccion] if 'Elecciones' in self.df.columns else self.df

        if ambito == 'municipio':
            municipio = random.choice(self.municipios)
            datos = datos_eleccion[datos_eleccion['Municipio'] == municipio]
            texto_ambito = f"en el municipio {municipio}"

        elif ambito == 'provincia':
            provincia = random.choice(self.provincias)
            datos = datos_eleccion[datos_eleccion['Provincia'] == provincia]
            texto_ambito = f"en la provincia de {provincia}"

        if len(datos) == 0:
            return None

        # Seleccionar 2 partidos diferentes
        partido1, partido2 = random.sample(self.partidos, 2)
        col1, col2 = f'% {partido1}', f'% {partido2}'

        if col1 not in datos.columns or col2 not in datos.columns:
            return None

        valor1 = 100 * datos[col1].mean()
        valor2 = 100 * datos[col2].mean()
        diferencia = abs(valor1 - valor2)

        pregunta = f"¿Qué diferencia hay entre {partido1} y {partido2} {texto_ambito} en {eleccion}?"

        respuesta = f"En la elección de {eleccion} {texto_ambito}, el partido {partido1} obtuvo el {valor1:.2f}%, y el partido {partido2} obtuvo el {valor2:.2f}%. "
        respuesta += f"La diferencia es de {diferencia:.2f} puntos porcentuales "
        respuesta += f"a favor de {partido1 if valor1 > valor2 else partido2}."

        return {
            "instruction": pregunta,
            "input": f"Año: {eleccion}, Partidos: {partido1} y {partido2}, Ámbito: {ambito}",
            "output": respuesta,
            "metadata": {
                "tipo": "diferencia_partidos",
                "partidos": [partido1, partido2],
                "ambito": ambito,
                "año": eleccion,
                "valores": [float(valor1), float(valor2)]
            }
        }

    def generar_correlacion_simple(self) -> Optional[Dict]:
        """
        Tipo 5: Identificar correlaciones simples entre variables
        """
        # Variables para correlacionar
        variable_electoral = random.choice(self.columnas_electorales)
        variable_socio = random.choice(self.variables_socio)

        # Limitar a municipios con datos
        municipios_con_datos = []
        for m in random.sample(self.municipios, min(50, len(self.municipios))):
            datos_mun = self.df[self.df['Municipio'] == m]
            if variable_electoral in datos_mun.columns and variable_socio in datos_mun.columns:
                if not datos_mun[variable_electoral].isna().all() and not datos_mun[variable_socio].isna().all():
                    municipios_con_datos.append(m)

        if len(municipios_con_datos) < 5:
            return None

        # Calcular correlación simple
        correlaciones = []
        for m in municipios_con_datos[:10]:  # Muestra de 10 municipios
            datos_mun = self.df[self.df['Municipio'] == m]
            valor_electoral = datos_mun[variable_electoral].mean()
            valor_socio = datos_mun[variable_socio].mean()
            correlaciones.append((valor_electoral, valor_socio))

        # Determinar tendencia
        electoral_vals = [c[0] for c in correlaciones]
        socio_vals = [c[1] for c in correlaciones]

        # Clasificación simple
        avg_electoral = np.mean(electoral_vals)
        avg_socio = np.mean(socio_vals)

        # Municipios "altos" en socio
        altos_socio = [c for c in correlaciones if c[1] > avg_socio]
        bajos_socio = [c for c in correlaciones if c[1] <= avg_socio]

        avg_electoral_altos = np.mean([c[0] for c in altos_socio]) if altos_socio else 0
        avg_electoral_bajos = np.mean([c[0] for c in bajos_socio]) if bajos_socio else 0

        tendencia = "positiva" if avg_electoral_altos > avg_electoral_bajos else "negativa"

        pregunta = f"¿Qué relación hay entre {variable_electoral} y {variable_socio} en los municipios?"

        respuesta = f"Analizando 10 municipios, los que tienen mayor {variable_socio} "
        respuesta += f"tienen un {variable_electoral} de {avg_electoral_altos:.1f}%, "
        respuesta += f"mientras que los de menor {variable_socio} tienen {avg_electoral_bajos:.1f}%. "
        respuesta += f"Esto sugiere una relación {tendencia}: a mayor {variable_socio}, "
        respuesta += f"{'mayor' if tendencia == 'positiva' else 'menor'} {variable_electoral}."

        return {
            "instruction": pregunta,
            "input": f"Variables: {variable_electoral} y {variable_socio}",
            "output": respuesta,
            "metadata": {
                "tipo": "correlacion_simple",
                "variables": [variable_electoral, variable_socio],
                "tendencia": tendencia,
                "muestra": len(correlaciones)
            }
        }

    def generar_ranking_top_n(self) -> Optional[Dict]:
        """
        Tipo 6: Top N municipios/provincias por alguna variable
        """
        variable = random.choice(self.columnas_electorales)
        n = random.choice([3, 5, 10])
        ambito = random.choice(['municipio', 'provincia'])
        eleccion = random.choice(self.elecciones) if 'Elecciones' in self.df.columns else 'Abr19'

        datos_eleccion = self.df[self.df['Elecciones'] == eleccion] if 'Elecciones' in self.df.columns else self.df

        # datos_año = self.df[self.df['Año'] == año] if 'Año' in self.df.columns else self.df

        if ambito == 'municipio':
            # Agrupar por municipio
            agrupado = datos_eleccion.groupby('Municipio')[variable].mean().reset_index()
            agrupado = agrupado.sort_values(by=variable, ascending=False).head(n)

            top_n = agrupado['Municipio'].tolist()
            valores = agrupado[variable].tolist()

            pregunta = f"¿Cuáles son los {n} municipios con mayor {variable} en la elección de {eleccion}?"

            respuesta = f"Los {n} municipios con mayor {variable} en la elección de {eleccion} son: "
            for i, (mun, val) in enumerate(zip(top_n, valores), 1):
                respuesta += f"{i}. {mun} ({100 * val:.2f}%), "
            respuesta = respuesta.rstrip(", ")

        if ambito == 'provincia':
              # Agrupar por provincia
            agrupado = datos_eleccion.groupby('Provincia')[variable].mean().reset_index()
            agrupado = agrupado.sort_values(by=variable, ascending=False).head(n)

            top_n = agrupado['Provincia'].tolist()
            valores = agrupado[variable].tolist()

            pregunta = f"¿Cuáles son las {n} provincias con mayor {variable} en la elección de {eleccion}?"

            respuesta = f"Las {n} provincias con mayor {variable} en la elección de {eleccion} son: "
            for i, (mun, val) in enumerate(zip(top_n, valores), 1):
                respuesta += f"{i}. {mun} ({val:.2f}%), "
            respuesta = respuesta.rstrip(", ")

        return {
            "instruction": pregunta,
            "input": f"Año: {eleccion}, Variable: {variable}, Top: {n}",
            "output": respuesta,
            "metadata": {
                "tipo": "ranking_top_n",
                "variable": variable,
                "n": n,
                "ambito": ambito,
                "año": eleccion,
                "ranking": list(zip(top_n, valores))
            }
        }

    def generar_cambio_porcentual(self) -> Optional[Dict]:
        """
        Tipo 7: Cambio porcentual (no solo diferencia absoluta)
        """
        if len(self.elecciones) < 2:
            return None

        municipio = random.choice(self.municipios)
        partido = random.choice(self.partidos)
        eleccion1, eleccion2 = random.sample(self.elecciones, 2)

        datos_eleccion1 = self.df[(self.df['Municipio'] == municipio) & (self.df['Elecciones'] == eleccion1)]
        datos_eleccion2 = self.df[(self.df['Municipio'] == municipio) & (self.df['Elecciones'] == eleccion2)]

        if len(datos_eleccion1) == 0 or len(datos_eleccion2) == 0:
            return None

        columna = f'% {partido}'
        if columna not in datos_eleccion1.columns or columna not in datos_eleccion2.columns:
            return None

        valor1 = 100 * datos_eleccion1[columna].mean()
        valor2 = 100 * datos_eleccion2[columna].mean()

        if valor1 == 0:
            return None

        cambio_porcentual = ((valor2 - valor1) / valor1) * 100

        pregunta = f"¿Cuál fue el cambio porcentual del voto a {partido} en {municipio} entre las elecciones de {eleccion1} y {eleccion2}?"

        respuesta = f"El {partido} en {municipio} pasó de {valor1:.2f}% en porcentaje de voto en la elección {eleccion1} al {valor2:.2f}% en {eleccion2}. "
        respuesta += f"Esto representa un cambio del {cambio_porcentual:+.1f}% "
        respuesta += f"({'de incremento' if cambio_porcentual > 0 else ' de disminución'} porcentual)."

        return {
            "instruction": pregunta,
            "input": f"Municipio: {municipio}, Partido: {partido}, Años: {eleccion1}->{eleccion2}",
            "output": respuesta,
            "metadata": {
                "tipo": "cambio_porcentual",
                "municipio": municipio,
                "partido": partido,
                "años": [eleccion1, eleccion2],
                "cambio_porcentual": float(cambio_porcentual)
            }
        }

    def generar_agregacion(self) -> Optional[Dict]:
        """
        Tipo 8: Sumas, conteos, proporciones
        """
        operacion = random.choice(['suma', 'conteo', 'proporcion'])
        eleccion = random.choice(self.elecciones)

        datos_eleccion = self.df[self.df['Elecciones'] == eleccion] if 'Elecciones' in self.df.columns else self.df

        if operacion == 'suma':
            provincia = random.choice(self.provincias)
            datos_prov = datos_eleccion[datos_eleccion['Provincia'] == provincia]

            total_votos = datos_prov['Votos_Total'].sum() if 'Votos_Total' in datos_prov.columns else 0

            pregunta = f"¿Cuántos votos totales hubo en la provincia de {provincia} en {eleccion}?"
            respuesta = f"En la provincia de {provincia} en {eleccion} hubo {total_votos:,.0f} votos totales."

        elif operacion == 'conteo':
            municipio = random.choice(self.municipios)
            datos_mun = datos_eleccion[datos_eleccion['Municipio'] == municipio]

            num_secciones = len(datos_mun)

            pregunta = f"¿Cuántas secciones electorales tiene el municipio de {municipio} en {eleccion}?"
            respuesta = f"El municipio de {municipio} tiene {num_secciones} secciones electorales en {eleccion}."


        elif operacion == 'proporcion':
            comunidad = random.choice(self.comunidades)
            datos_comun = datos_eleccion[datos_eleccion['CCAA'] == comunidad]
            provincia_1 = random.choice(datos_comun['Provincia'].unique())
            datos_prov = datos_eleccion[datos_eleccion['Provincia'] == provincia_1]
            municipio_1 = random.choice(datos_prov['Municipio'].unique())
            datos_mun = datos_eleccion[datos_eleccion['Municipio'] == municipio_1]

            total_censo_comunidad = datos_comun['Censo_Esc'].sum() if 'Censo_Esc' in datos_comun.columns else 0
            total_censo_provincia = datos_prov['Censo_Esc'].sum() if 'Censo_Esc' in datos_prov.columns else 0
            total_censo_municipio = datos_mun['Censo_Esc'].sum() if 'Censo_Esc' in datos_mun.columns else 0

            proporcion_comunidad = (total_censo_comunidad / total_censo_provincia) * 100
            proporcion_municipio = (total_censo_municipio / total_censo_provincia) * 100

            pregunta = f"¿Cuál es la proporción de censo electoral del municipio de {municipio_1} en la provincia de {provincia_1} y en la comunidad de {comunidad}?"
            respuesta = f"El municipio de {municipio_1} representa el {proporcion_municipio:.1f}% del censo en la provincia de {provincia_1} y el {proporcion_comunidad:.1f}% del censo de la comunidad {comunidad}"

        return {
            "instruction": pregunta,
            "input": f"Elección: {eleccion}, Operación: {operacion}",
            "output": respuesta,
            "metadata": {
                "tipo": "agregacion",
                "operacion": operacion,
                "año": eleccion
            }
        }

    def generar_ejemplo_nivel2(self) -> Optional[Dict]:
        """
        Selecciona aleatoriamente un tipo de pregunta del nivel 2
        """
        generadores = [
            self.generar_evolucion_temporal,
            self.generar_comparacion_geografica,
            self.generar_operacion_estadistica,
            self.generar_diferencia_partidos,
            self.generar_correlacion_simple,
            self.generar_ranking_top_n,
            self.generar_cambio_porcentual,
            self.generar_agregacion
        ]

        # Ponderar algunos tipos más que otros
        pesos = [1.2, 1.0, 1.0, 1.0, 0.8, 0.8, 1.0, 0.8]

        # Intentar hasta 3 veces si un generador falla
        for _ in range(3):
            generador = random.choices(generadores, weights=pesos, k=1)[0]
            # print(generador)
            ejemplo = generador()
            if ejemplo is not None:
                return ejemplo

        return None


Aquí tenemos la función que genera el dataset de Nivel 2, que básicamente activa la clase anterior (más bien llama al método que genera preguntas/respuestas) que hemos visto antes.

In [ ]:

# ============================================================================
# FUNCIÓN PRINCIPAL PARA GENERAR DATASET COMPLETO
# ============================================================================

def generar_dataset_nivel2(df_path: str, n_ejemplos: int = 600, output_path: str = "dataset_nivel2.jsonl"):
    """
    Genera un dataset completo de preguntas nivel 2

    Args:
        df_path: Ruta al archivo CSV
        n_ejemplos: Número de ejemplos a generar
        output_path: Ruta donde guardar el dataset
    """
    # Cargar datos
    print(f"Cargando datos desde {df_path}...")

    strings = {'Sección' : 'str', 'cod_ccaa' : 'str', 'cod_prov' : 'str', 'cod_mun' : 'str', 'cod_sec' : 'str'}


    df = pd.read_csv(df_path, dtype = strings)

    # Si tienes múltiples datasets, añadir columna 'Año'
    # Ejemplo: df['Año'] = 2019  # O leer de nombre de archivo

    # Inicializar generador
    generador = GeneradorPreguntasNivel2(df)

    # Generar ejemplos
    dataset = []
    print(f"Generando {n_ejemplos} ejemplos nivel 2...")

    for i in range(n_ejemplos):
        if i % 25 == 0:
            print(f"  Progreso: {i}/{n_ejemplos}")

        ejemplo = generador.generar_ejemplo_nivel2()
        if ejemplo is not None:
            dataset.append(ejemplo)

    print(f"Generados {len(dataset)} ejemplos válidos.")

    # Estadísticas por tipo
    tipos = [e['metadata']['tipo'] for e in dataset]
    print("\nDistribución por tipo:")
    for tipo in set(tipos):
        count = tipos.count(tipo)
        print(f"  {tipo}: {count} ({count/len(dataset)*100:.1f}%)")

    # Guardar en formato JSONL (una línea por ejemplo)
    print(f"\nGuardando en {output_path}...")
    with open(output_path, 'w', encoding='utf-8') as f:
        for ejemplo in dataset:
            f.write(json.dumps(ejemplo, ensure_ascii=False) + '\n')

    # También guardar versión para inspección
    with open(output_path.replace('.jsonl', '_pretty.json'), 'w', encoding='utf-8') as f:
        json.dump(dataset, f, ensure_ascii=False, indent=2)

    print("¡Dataset generado exitosamente!")
    return dataset


Finalmente este el la función inicial, que genera un dataset de 600 preguntas/respuestas del Nivel 2.

In [ ]:

# ============================================================================
# EJEMPLO DE USO
# ============================================================================

if __name__ == "__main__":
    # Configuración
    RUTA_CSV = "/content/drive/MyDrive/Practica_LLM_Engineering_25/df_elecciones_19.csv"  # Tu archivo
    NUM_EJEMPLOS = 600  # Para nivel 2
    RUTA_SALIDA = "/content/drive/MyDrive/Practica_LLM_Engineering_25/dataset_preguntas_nivel2.jsonl"

    # Ejecutar generación
    dataset = generar_dataset_nivel2(
        df_path=RUTA_CSV,
        n_ejemplos=NUM_EJEMPLOS,
        output_path=RUTA_SALIDA
    )

    # Mostrar algunos ejemplos
    print("\n\n--- EJEMPLOS GENERADOS ---")
    for i, ejemplo in enumerate(dataset[:3]):
        print(f"\nEjemplo {i+1}:")
        print(f"Pregunta: {ejemplo['instruction']}")
        print(f"Respuesta: {ejemplo['output'][:150]}...")
        print(f"Tipo: {ejemplo['metadata']['tipo']}")

Cargando datos desde /content/drive/MyDrive/Practica_LLM_Engineering_25/df_elecciones_19.csv...
Generando 600 ejemplos nivel 2...
  Progreso: 0/600
  Progreso: 25/600
  Progreso: 50/600
  Progreso: 75/600
  Progreso: 100/600
  Progreso: 125/600
  Progreso: 150/600
  Progreso: 175/600
  Progreso: 200/600
  Progreso: 225/600
  Progreso: 250/600
  Progreso: 275/600
  Progreso: 300/600
  Progreso: 325/600
  Progreso: 350/600
  Progreso: 375/600
  Progreso: 400/600
  Progreso: 425/600
  Progreso: 450/600
  Progreso: 475/600
  Progreso: 500/600
  Progreso: 525/600
  Progreso: 550/600
  Progreso: 575/600
Generados 600 ejemplos válidos.

Distribución por tipo:
  agregacion: 53 (8.8%)
  operacion_estadistica: 91 (15.2%)
  comparacion_geografica: 76 (12.7%)
  evolucion_temporal: 100 (16.7%)
  correlacion_simple: 78 (13.0%)
  diferencia_partidos: 66 (11.0%)
  cambio_porcentual: 59 (9.8%)
  ranking_top_n: 77 (12.8%)

Guardando en /content/drive/MyDrive/Practica_LLM_Engineering_25/dataset_preguntas

Este es el resumen de las pesos de cada uno de los templates (métodos) que aspiramos a crear, que no se diferencias demasiado de lo obtenido.

In [ ]:
distribucion_recomendada = {
    "evolucion_temporal": 90,      # 15%
    "comparacion_geografica": 90,  # 15%
    "operacion_estadistica": 90,   # 15%
    "diferencia_partidos": 90,     # 15%
    "correlacion_simple": 60,      # 10%
    "ranking_top_n": 60,           # 10%
    "cambio_porcentual": 60,       # 10%
    "agregacion": 60               # 10%
}